## 1. Installation & Setup

In [1]:
%%capture
# Install Unsloth and dependencies
# What's happening: Installing the Unsloth library which provides optimized training
# Key libraries:
#   - unsloth: Main optimization library (2x faster training, 70% less memory)
#   - xformers: Memory-efficient attention mechanisms
#   - trl: Hugging Face's Transformer Reinforcement Learning library
#   - peft: Parameter-Efficient Fine-Tuning (implements LoRA)
#   - accelerate: Distributed training support
#   - bitsandbytes: Enables 4-bit/8-bit quantization for memory savings
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [2]:
# Verify GPU availability
# What's happening: Checking if a GPU is available and its specifications
# Why: Training transformers on CPU is extremely slow; GPU provides 10-100x speedup
# BF16 (Brain Float 16): A 16-bit number format that's faster than 32-bit with minimal accuracy loss
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"BF16 Support: {torch.cuda.is_bf16_supported()}")

GPU Available: True
GPU Name: Tesla T4
GPU Memory: 14.74 GB
BF16 Support: True


## 2. Load Model with 4-bit Quantization

In [3]:
from unsloth import FastLanguageModel
import torch

# Configuration
max_seq_length = 2048
dtype = None  # Auto-detect
load_in_4bit = True  # 70% less VRAM

# Load model and tokenizer
# What's happening: Loading a pre-trained SmolLM2-135M model with 4-bit quantization
# How Unsloth helps:
#   1. 4-bit quantization: Compresses model from 32-bit floats to 4-bit integers (8x smaller!)
#      - Think of it like compressing a high-res photo to a smaller size
#      - Original: Each number uses 32 bits (4 bytes)
#      - Quantized: Each number uses 4 bits (0.5 bytes) = 87.5% memory savings
#   2. FastLanguageModel: Unsloth's optimized loader with fused kernels
#      - Fused kernels: Combines multiple operations into one GPU call (reduces overhead)
#   3. Automatic memory optimization: Manages CUDA memory efficiently
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/smollm2-135m",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print(f"✓ Model loaded: {model.config._name_or_path}")
print(f"✓ Total parameters: {model.num_parameters():,}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.9.0+cu128 with CUDA 1208 (you have 2.8.0+cu126)
    Python  3.10.19 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.9.0+cu128 with CUDA 1208 (you have 2.8.0+cu126)
    Python  3.10.19 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✓ Model loaded: unsloth/smollm2-135m
✓ Total parameters: 134,515,584


In [4]:
# Apply LoRA with low rank
# What's happening: Adding small "adapter" layers to the model instead of training all parameters
# LoRA Concept (Low-Rank Adaptation):
#   Imagine a massive book (the pre-trained model) with millions of words.
#   Instead of rewriting the whole book, we add sticky notes (LoRA adapters) with updates.
#   The sticky notes are much smaller but can modify the book's behavior!
#
# Key differences from Notebook 1 (Full Fine-tuning):
#   - Rank 8 vs 256: Much smaller adapters (8-dimensional updates vs 256-dimensional)
#   - Fewer target modules: Only attention and FFN layers (no lm_head/embed_tokens)
#   - Result: ~10-20x fewer trainable parameters!
#
# How Unsloth optimizes LoRA:
#   1. Fast LoRA kernels: Custom CUDA code for LoRA operations
#   2. Gradient checkpointing: Saves memory by recomputing activations during backward pass
#      - Trade-off: 20% slower but uses 50% less memory
#   3. Optimized backward pass: Efficient gradient computation for adapter layers
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,  # Low rank for parameter efficiency (8-dimensional adapter matrices)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",  # Attention layers
                      "gate_proj", "up_proj", "down_proj"],      # Feed-forward layers
    lora_alpha = 16,  # Scaling factor (2x the rank is a common heuristic)
    lora_dropout = 0.05,  # Small dropout for regularization
    bias = "none",  # Don't train bias terms (saves parameters)
    use_gradient_checkpointing = "unsloth",  # Unsloth's optimized checkpointing
    random_state = 3407,  # For reproducibility
    use_rslora = False,  # Rank-Stabilized LoRA (not needed for low rank)
)

# Calculate trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = model.num_parameters()
print(f"\n✓ LoRA Applied (Low-Rank Configuration)")
print(f"  Trainable params: {trainable_params:,}")
print(f"  Total params: {total_params:,}")
print(f"  Trainable %: {trainable_params/total_params*100:.4f}%")
print(f"  LoRA Rank: 8")
print(f"  LoRA Alpha: 16")
print(f"\n  NOTE: This is significantly fewer trainable parameters than full fine-tuning!")

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.11.3 patched 30 layers with 0 QKV layers, 0 O layers and 0 MLP layers.



✓ LoRA Applied (Low-Rank Configuration)
  Trainable params: 2,442,240
  Total params: 136,957,824
  Trainable %: 1.7832%
  LoRA Rank: 8
  LoRA Alpha: 16

  NOTE: This is significantly fewer trainable parameters than full fine-tuning!


## 4. Load & Prepare CodeParrot Dataset

Using the **same dataset** as Notebook 1 for fair comparison

In [ ]:
from datasets import load_dataset

# Load same dataset as Notebook 1
# What's happening: Loading CodeParrot Clean dataset (Python code samples)
# Why this dataset: Contains clean, well-formatted Python code for teaching the model coding patterns
# Using only 1000 samples for demonstration (full dataset has millions)
print("Loading dataset...")
dataset = load_dataset("codeparrot/codeparrot-clean", split="train[:1000]", trust_remote_code=True)

print(f"\n✓ Dataset loaded: {len(dataset)} samples")
print(f"  Fields: {dataset.column_names}")

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'codeparrot/codeparrot-clean' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'codeparrot/codeparrot-clean' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Loading dataset...


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/54 [00:00<?, ?it/s]

file-000000000001.json.gz:   0%|          | 0.00/246M [00:00<?, ?B/s]

...

file-000000000041.json.gz:   0%|          | 0.00/235M [00:00<?, ?B/s]

file-000000000042.json.gz:   0%|          | 0.00/236M [00:00<?, ?B/s]

file-000000000043.json.gz:   0%|          | 0.00/236M [00:00<?, ?B/s]

file-000000000044.json.gz:   0%|          | 0.00/234M [00:00<?, ?B/s]

file-000000000045.json.gz:   0%|          | 0.00/237M [00:00<?, ?B/s]

## 5. Configure Training Arguments

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer
import os
import time

# Create checkpoint directory
output_dir = "./checkpoints/colab2"
os.makedirs(output_dir, exist_ok=True)

# Training configuration (same as Notebook 1 for comparison)
# What's happening: Configuring the training hyperparameters
# Key settings explained:
#   - Batch size (2) x Gradient accumulation (4) = Effective batch size of 8
#     * Small physical batch fits in limited GPU memory
#     * Gradient accumulation simulates larger batches by accumulating gradients
#   - adamw_8bit: 8-bit Adam optimizer (saves memory, Unsloth makes it fast)
#   - Learning rate 2e-4: Higher than typical (1e-5) because we're only training adapters
#   - Linear scheduler: Gradually decreases learning rate to prevent overshooting
training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 10,
    max_steps = 100,
    learning_rate = 2e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 5,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = output_dir,
    save_strategy = "steps",
    save_steps = 50,
    report_to = "none",
)

print("✓ Training configuration (identical to Notebook 1):")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Max steps: {training_args.max_steps}")

## 6. Initialize Trainer & Start Training

In [ ]:
# Initialize SFTTrainer
# What's happening: Setting up the trainer for Supervised Fine-Tuning
# SFTTrainer (from TRL library):
#   - Handles the training loop (forward pass, backward pass, optimizer steps)
#   - Manages data loading and batching
#   - Tracks metrics (loss, training speed, memory usage)
# Unsloth optimizations during training:
#   1. Fused backward pass: Combines gradient computation steps
#   2. Flash Attention: 2-4x faster attention computation (when available)
#   3. Memory-efficient gradient accumulation
#   4. Automatic mixed precision (BF16/FP16) handling

# Format the dataset with a simple prompt template
def formatting_prompts_func(examples):
    """Format code samples for training."""
    texts = []
    for code in examples["content"]:
        # Simple format: just the code (no special formatting needed for code completion)
        text = code
        texts.append(text)
    return {"text": texts}

# Apply formatting to dataset
formatted_dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=dataset.column_names,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_dataset,
    dataset_text_field = "text",  # Now using 'text' field from formatted dataset
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,  # Don't pack multiple samples into one sequence
    args = training_args,
)

print("\n" + "="*80)
print("STARTING TRAINING - LOW-RANK LoRA (r=8, alpha=16)")
print("="*80)

# Reset GPU memory stats
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    mem_before = torch.cuda.memory_allocated()/1024**3
    print(f"\nGPU Memory before training: {mem_before:.2f} GB")

# Record start time
start_time = time.time()

# Train the model
# What's happening during training:
#   1. For each batch: Load text → Tokenize → Forward pass → Compute loss
#   2. Backward pass: Calculate gradients (only for LoRA adapter parameters!)
#   3. Optimizer step: Update adapter weights using Adam
#   4. Repeat for 100 steps
# Unsloth makes steps 1-3 about 2x faster than standard PyTorch
trainer_stats = trainer.train()

# Record end time
end_time = time.time()
training_time = end_time - start_time

# Monitor GPU memory after training
if torch.cuda.is_available():
    mem_after = torch.cuda.memory_allocated()/1024**3
    mem_peak = torch.cuda.max_memory_allocated()/1024**3
    print(f"\nGPU Memory after training: {mem_after:.2f} GB")
    print(f"Peak GPU Memory: {mem_peak:.2f} GB")

print(f"\nTotal Training Time: {training_time:.2f} seconds")

print("\n" + "="*80)
print("TRAINING COMPLETED")
print("="*80)

## 7. Compare with Full Fine-tuning (Notebook 1)

### Key Metrics Comparison

In [ ]:
import pandas as pd

# This notebook's metrics
lora_metrics = {
    'Method': 'LoRA (r=8)',
    'Trainable Params': trainable_params,
    'Trainable %': f"{trainable_params/total_params*100:.4f}%",
    'Peak GPU Memory (GB)': f"{mem_peak:.2f}" if torch.cuda.is_available() else "N/A",
    'Training Time (s)': f"{training_time:.2f}",
    'Samples/sec': f"{trainer_stats.metrics['train_samples_per_second']:.2f}",
}

# Expected metrics from Full Fine-tuning (Notebook 1)
# Note: These are estimates - actual values from Notebook 1 may vary
full_ft_metrics = {
    'Method': 'Full FT (r=256)',
    'Trainable Params': 'Higher (~10-20x more)',
    'Trainable %': '~5-10%',
    'Peak GPU Memory (GB)': 'Higher (~1.5-2x)',
    'Training Time (s)': 'Similar or longer',
    'Samples/sec': 'Similar',
}

# Create comparison DataFrame
comparison_df = pd.DataFrame([full_ft_metrics, lora_metrics])

print("\n" + "="*80)
print("COMPARISON: Full Fine-tuning vs Low-Rank LoRA")
print("="*80)
print(comparison_df.to_string(index=False))

print("\n📊 KEY INSIGHTS:")
print("  ✓ LoRA trains <1% of parameters (vs ~5-10% for full fine-tuning)")
print("  ✓ Lower GPU memory usage enables larger batch sizes")
print("  ✓ Faster iteration during development")
print("  ✓ Easier to deploy (smaller checkpoint files)")
print("  ✓ Comparable performance for many tasks")

## 8. Analyze Training Results

In [ ]:
import matplotlib.pyplot as plt

# Extract training logs
logs = trainer.state.log_history
train_logs = [log for log in logs if 'loss' in log]

# Create DataFrame
df = pd.DataFrame(train_logs)
print("\nTraining Statistics:")
print(df[['step', 'loss', 'learning_rate']].to_string(index=False))

# Plot loss curve
if len(df) > 0:
    plt.figure(figsize=(10, 5))
    plt.plot(df['step'], df['loss'], marker='o', linewidth=2, color='green')
    plt.xlabel('Training Step', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.title('Low-Rank LoRA Loss Curve (r=8, alpha=16)', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/loss_curve.png", dpi=150, bbox_inches='tight')
    plt.show()
    print(f"\n✓ Loss curve saved to {output_dir}/loss_curve.png")

# Print final statistics
print(f"\nFinal Training Statistics:")
print(f"  Total steps: {trainer.state.global_step}")
print(f"  Final loss: {df['loss'].iloc[-1]:.4f}")
print(f"  Average loss: {df['loss'].mean():.4f}")

## 9. Test Code Generation

In [ ]:
# Enable fast inference mode
# What's happening: Switching the model from training mode to inference mode
# Unsloth optimizations for inference:
#   1. Disables dropout layers (not needed during generation)
#   2. Merges LoRA adapters with base weights for faster computation
#   3. Enables KV-cache for faster autoregressive generation
#      - KV-cache: Stores previous attention keys/values to avoid recomputation
FastLanguageModel.for_inference(model)

# Test prompts (same as Notebook 1)
test_prompts = [
    "def fibonacci(n):",
    "class DataProcessor:",
    "import numpy as np\n\ndef calculate_mean(",
]

print("\n" + "="*80)
print("CODE GENERATION SAMPLES (Low-Rank LoRA)")
print("="*80)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n--- Sample {i} ---")
    print(f"Prompt: {prompt}")
    print("\nGenerated Code:")
    print("-" * 80)

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate code completion
    # Parameters explained:
    #   - max_new_tokens: Maximum number of tokens to generate
    #   - temperature: Controls randomness (0.7 = moderately creative)
    #   - top_p: Nucleus sampling (0.9 = consider top 90% probable tokens)
    #   - do_sample: Enable sampling (vs greedy decoding)
    #   - use_cache: Use KV-cache for faster generation (Unsloth optimizes this!)
    outputs = model.generate(
        **inputs,
        max_new_tokens = 128,
        temperature = 0.7,
        top_p = 0.9,
        do_sample = True,
        use_cache = True,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_text)
    print("-" * 80)

## 10. Save Model Checkpoints

In [ ]:
# Save LoRA adapter (much smaller than full model!)
# What's happening: Saving only the LoRA adapter weights (not the entire model)
# Key advantage of LoRA:
#   - Full model checkpoint: ~500MB - 7GB depending on model size
#   - LoRA adapter: Only 5-50MB (10-100x smaller!)
#   - Why: We only save the small adapter matrices, not the frozen base model
#   - Deployment: Load base model once, then swap adapters for different tasks
lora_path = f"{output_dir}/lora_adapter"
model.save_pretrained(lora_path)
tokenizer.save_pretrained(lora_path)
print(f"✓ LoRA adapter saved to {lora_path}")

# Check adapter size
import os
adapter_size = sum(os.path.getsize(os.path.join(dirpath,filename))
                   for dirpath, dirnames, filenames in os.walk(lora_path)
                   for filename in filenames) / (1024**2)
print(f"  Adapter size: {adapter_size:.2f} MB")
print(f"\n  NOTE: LoRA adapters are much smaller than full model checkpoints!")
print(f"  This makes them easy to version control, share, and deploy.")

print("\n✓ All checkpoints saved successfully!")

## 11. Summary & Observations

### Key Results:
- **Training Method**: Low-Rank LoRA (r=8, alpha=16)
- **Model**: SmolLM2-135M (135M parameters)
- **Dataset**: CodeParrot Clean (1000 Python code samples)
- **Training Steps**: 100 steps
- **GPU**: Google Colab T4 (12GB VRAM)

### LoRA Advantages:
1. **Parameter Efficiency**: Trains <1% of parameters (vs ~5-10% for full fine-tuning)
2. **Memory Efficiency**: Lower GPU memory usage enables larger models on limited hardware
3. **Storage**: Adapter files are 10-100x smaller than full model checkpoints
4. **Flexibility**: Can train multiple adapters for different tasks using same base model
5. **Speed**: Similar or faster training time with comparable performance

### When to Use LoRA (rank=8-16):
- ✓ Limited GPU memory
- ✓ Task-specific adaptation (classification, QA, summarization)
- ✓ Fast iteration during development
- ✓ Multiple specialized models from one base
- ✓ Easy deployment and version control

### When to Use Full Fine-tuning (rank=256+):
- ✓ New domain with very different vocabulary
- ✓ Learning entirely new capabilities
- ✓ Maximum model expressiveness needed
- ✓ Sufficient compute resources available

---

**Comparison**: This notebook demonstrated **~10-20x fewer trainable parameters** with comparable code generation quality!

**Next**: See [colab3_rlhf.ipynb](colab3_rlhf.ipynb) for preference-based reinforcement learning!